# 数据包采集与标注工具

本工具结合 Scapy（或 Kamene）、Wireshark 和 Label Studio，实现网络数据包的自动采集、解析和标注：

- **采集**：自动抓取网络数据包并保存为 pcap 文件，同时记录采集参数。
- **解析**：将 pcap 文件解析为结构化 JSON，便于后续标注和分析。
- **标注**：可将 JSON 转为 CSV，方便在 Label Studio 等工具中进行批量标注。

整体流程：采集数据包 → 保存为 pcap → 解析为 JSON → 可选导出为 CSV → 标注

In [ ]:
from kamene.all import *
import time
import json

def capture_packets_with_params(count=100, iface=None, filter_rule='', duration=10):
    """抓包并保存参数记录"""
    ts = time.strftime('%Y%m%d_%H%M%S')
    pcap_filename = f'{ts}.pcap'
    params_filename = f'{ts}_params.json'
    print(f'开始抓包，文件名: {pcap_filename}')
    packets = sniff(count=count, iface=iface, filter=filter_rule, timeout=duration)
    wrpcap(pcap_filename, packets)
    params = {
        'iface': iface,
        'filter_rule': filter_rule,
        'count': count,
        'duration': duration,
        'pcap_filename': pcap_filename,
        'timestamp': ts
    }
    with open(params_filename, 'w', encoding='utf-8') as f:
        json.dump(params, f, ensure_ascii=False, indent=2)
    print(f'参数记录已保存到 {params_filename}')

## 采集环节

使用 `capture_packets_with_params` 自动抓取网络数据包并保存为 pcap 文件，同时生成采集参数记录（JSON）。  
示例：抓取 TCP 包，网卡为 eth0，持续 10 秒  
`capture_packets_with_params(count=100, iface='eth0', filter_rule='tcp', duration=10)`

In [ ]:
# 示例：抓取TCP包，网卡为 eth0，持续10秒
# capture_packets_with_params(count=100, iface='eth0', filter_rule='tcp', duration=10)

## 解析环节

使用 `parse_pcap_to_json` 解析 pcap 文件，提取关键信息并导出为结构化 JSON。  
示例：`parse_pcap_to_json('20240724_1000.pcap', '20240724_1000.json')`

In [ ]:
def parse_pcap_to_json(pcap_file, json_file):
    """解析pcap文件，输出结构化JSON"""
    packets = rdpcap(pcap_file)
    result = []
    for idx, pkt in enumerate(packets):
        info = {
            'id': idx,
            'timestamp': pkt.time,
            'src_ip': pkt[IP].src if IP in pkt else '',
            'dst_ip': pkt[IP].dst if IP in pkt else '',
            'src_port': pkt.sport if hasattr(pkt, 'sport') else '',
            'dst_port': pkt.dport if hasattr(pkt, 'dport') else '',
            'protocol': pkt[IP].proto if IP in pkt else '',
            'payload_summary': str(pkt.payload)[:50]
        }
        result.append(info)
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    print(f'已导出结构化数据到 {json_file}')

# 示例：parse_pcap_to_json('20240724_1000.pcap', '20240724_1000.json')

可选  csv导出
## 标注环节（可选）

如需在 Label Studio 等工具中进行批量标注，可使用 `json_to_csv` 将 JSON 数据转为 CSV 格式。  
示例：`json_to_csv('20240724_1000.json', '20240724_1000.csv')`

In [ ]:
# 将 JSON 结构化数据导出为 CSV，便于 Label Studio 标注
import pandas as pd

def json_to_csv(json_file, csv_file):
    """将JSON结构化数据导出为CSV"""
    import json
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    df = pd.DataFrame(data)
    df.to_csv(csv_file, index=False)
    print(f'已导出数据包信息到 {csv_file}')